In [1]:
import numpy as np
import pandas as pd
import warnings

import matplotlib.pyplot as plt
plt.style.use('ggplot')
import seaborn as sns

import nltk
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords 
from nltk.stem import WordNetLemmatizer 
from wordcloud import WordCloud

from sklearn.pipeline import Pipeline

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, classification_report
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB, BernoulliNB, GaussianNB
from collections import OrderedDict

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


CoAID News Articles

In [3]:
path = "/content/drive/MyDrive/"
#News Articles
df1 = pd.read_csv(path+'NewsRealCOVID-19-05.csv')
df2 = pd.read_csv(path+'NewsFakeCOVID-19-05.csv')
df1['label'] = 1
df2['label'] = 0

df_May_News = pd.concat([df1, df2]).reset_index(drop=True)
#shuffle
df_May_News = df_May_News.sample(frac=1)
#Remove NaN row if its content is NaN
df_May_News = df_May_News[df_May_News['content'].notna()]
print('May News Articles shape: ',df_May_News.shape)

May News Articles shape:  (1522, 16)


In [4]:
df3 = pd.read_csv(path+'NewsRealCOVID-19-07.csv')
df4 = pd.read_csv(path+'NewsFakeCOVID-19-07.csv')
df3['label'] = 1
df4['label'] = 0

df_July_News = pd.concat([df3, df4]).reset_index(drop=True)
#shuffle
df_July_News = df_July_News.sample(frac=1)
#Remove NaN row if its content is NaN
df_July_News = df_July_News[df_July_News['content'].notna()]
print('July News Articles shape: ',df_July_News.shape)

July News Articles shape:  (1212, 16)


In [17]:
df5 = pd.read_csv(path+'NewsRealCOVID-19-09.csv')
df6 = pd.read_csv(path+'NewsFakeCOVID-19-09.csv')
df5['label'] = 1
df6['label'] = 0

df_Sept_News = pd.concat([df5, df6]).reset_index(drop=True)
#shuffle
df_Sept_News = df_Sept_News.sample(frac=1)
#Remove NaN row if its content is NaN
df_Sept_News = df_Sept_News[df_Sept_News['content'].notna()]
print('Sept News Articles shape: ',df_Sept_News.shape)

Sept News Articles shape:  (847, 16)


In [5]:
df7 = pd.read_csv(path+'NewsRealCOVID-19-11.csv')
df8 = pd.read_csv(path+'NewsFakeCOVID-19-11.csv')
df7['label'] = 1
df8['label'] = 0

df_Nove_News = pd.concat([df7, df8]).reset_index(drop=True)
#shuffle
df_Nove_News = df_Nove_News.sample(frac=1)
#Remove NaN row if its content is NaN
df_Nove_News = df_Nove_News[df_Nove_News['content'].notna()]
print('Nove News Articles shape: ',df_Nove_News.shape)

Nove News Articles shape:  (859, 16)


CoAID Tweet

In [6]:
df9 = pd.read_csv(path+'0501ClaimFakeCOVID-19_tweets_content_1.csv')
df10 = pd.read_csv(path+'0501ClaimRealCOVID-19_tweets_content_1.csv')
df11 = pd.read_csv(path+'0501NewsFakeCOVID-19_tweets_content.csv')
df12 = pd.read_csv(path+'0501NewsRealCOVID-19_tweets_content.csv')
df9['label'] = df11['label'] = 0
df10['label'] = df12['label'] = 1

#combine dataframes
df_May_Tweet = pd.concat([df9,df10,df11,df12]).reset_index(drop=True)
#shuffle
df_May_Tweet = df_May_Tweet.sample(frac=1)
#Remove NaN row if its tweet is NaN
df_May_Tweet = df_May_Tweet[df_May_Tweet['tweet_text'].notna()]
print('May Tweet shape: ',df_May_Tweet.shape)

May Tweet shape:  (95107, 3)


In [7]:
df13 = pd.read_csv(path+'0701ClaimFakeCOVID-19_tweets_content.csv')
df14 = pd.read_csv(path+'0701ClaimRealCOVID-19_tweets_content.csv')
df15 = pd.read_csv(path+'0701NewsFakeCOVID-19_tweets_content.csv')
df16 = pd.read_csv(path+'0701NewsRealCOVID-19_tweets_content.csv')
df13['label'] = df15['label'] = 0
df14['label'] = df16['label'] = 1

#combine dataframes
df_July_Tweet = pd.concat([df13,df14,df15,df16]).reset_index(drop=True)
#shuffle
df_July_Tweet = df_July_Tweet.sample(frac=1)
#Remove NaN row if its content is NaN
df_July_Tweet = df_July_Tweet[df_July_Tweet['tweet_text'].notna()]
print('July Tweet shape: ',df_July_Tweet.shape)

July Tweet shape:  (54098, 3)


In [8]:
df17 = pd.read_csv(path+'0901ClaimRealCOVID-19_tweets_content.csv')
df18 = pd.read_csv(path+'0901NewsFakeCOVID-19_tweets_content.csv')
df19 = pd.read_csv(path+'0901NewsRealCOVID-19_tweets_content.csv')
df18['label'] = 0
df17['label'] = df19['label'] = 1

#combine dataframes
df_Sept_Tweet = pd.concat([df17,df18,df19]).reset_index(drop=True)
#shuffle
df_Sept_Tweet = df_Sept_Tweet.sample(frac=1)
#Remove NaN row if its content is NaN
df_Sept_Tweet = df_Sept_Tweet[df_Sept_Tweet['tweet_text'].notna()]
print('Sept Tweet shape: ',df_Sept_Tweet.shape)

Sept Tweet shape:  (424, 3)


In [9]:
df20 = pd.read_csv(path+'1101ClaimRealCOVID-19_tweets.csv')
df21 = pd.read_csv(path+'1101NewsFakeCOVID-19_tweets.csv')
df22 = pd.read_csv(path+'1101NewsRealCOVID-19_tweets.csv')
df21['label'] = 0
df20['label'] = df22['label'] = 1

#remove repeated useless string - 'Feed generated with FetchRSS'
tweet_update = []

for tweet in df22['tweet_text']:
    if ('Feed generated with FetchRSS') in tweet:
        tweet_update.append(tweet[:len(tweet)-30])
    else:
        tweet_update.append(tweet)
df22['tweet_text'] = tweet_update

#combine dataframes
df_Nove_Tweet = pd.concat([df20,df21,df22]).reset_index(drop=True)
#shuffle
df_Nove_Tweet = df_Nove_Tweet.sample(frac=1)
#Remove NaN row if its content is NaN
df_Nove_Tweet = df_Nove_Tweet[df_Nove_Tweet['tweet_text'].notna()]
print('Nove Tweet shape: ',df_Nove_Tweet.shape)

Nove Tweet shape:  (195, 3)


CoAID Tweet+Reply

In [10]:
df23 = pd.read_csv(path+'0501ClaimFakeCOVID-19_tweets_replies_content_1.csv')
df24 = pd.read_csv(path+'0501ClaimRealCOVID-19_tweets_replies_content_1.csv')
df25 = pd.read_csv(path+'0501NewsFakeCOVID-19_tweets_replies_content.csv')
df26 = pd.read_csv(path+'0501NewsRealCOVID-19_tweets_replies_content.csv')
df23['label'] = df25['label'] = 0
df24['label'] = df26['label'] = 1

#combine dataframes
df_may_reply = pd.concat([df23,df24,df25,df26]).reset_index(drop=True)
#shuffle
df_may_reply = df_may_reply.sample(frac=1)   
#only keep tweet_reply and label columns
df_may_reply = df_may_reply.drop(['tweet_id', 'tweet_text'], axis=1)
#change df_reply column name to tweet_text in order to combine df_reply with df_tweets 
df_may_reply = df_may_reply.rename(columns={"tweet_reply": "tweet_text"})

df_May_Reply = pd.concat([df_May_Tweet,df_may_reply]).reset_index(drop=True)
print('May Tweets & Replies shape: ',df_May_Reply.shape)

May Tweets & Replies shape:  (163053, 3)


In [11]:
df27 = pd.read_csv(path+'0701ClaimFakeCOVID-19_tweets_replies_content.csv')
df28 = pd.read_csv(path+'0701ClaimRealCOVID-19_tweets_replies_content.csv')
df29 = pd.read_csv(path+'0701NewsFakeCOVID-19_tweets_replies_content.csv')
df30 = pd.read_csv(path+'0701NewsRealCOVID-19_tweets_replies_content.csv')
df27['label'] = df29['label'] = 0
df28['label'] = df30['label'] = 1

#combine dataframes
df_july_reply = pd.concat([df27,df28,df29,df30]).reset_index(drop=True)
#shuffle
df_july_reply = df_july_reply.sample(frac=1)   
#only keep tweet_reply and label columns
df_july_reply = df_july_reply.drop(['tweet_id', 'tweet_text'], axis=1)
#change df_reply column name to tweet_text in order to combine df_reply with df_tweets 
df_july_reply = df_july_reply.rename(columns={"tweet_reply": "tweet_text"})

df_July_Reply = pd.concat([df_July_Tweet,df_july_reply]).reset_index(drop=True)
print('July Tweets & Replies shape: ',df_July_Reply.shape)

July Tweets & Replies shape:  (101649, 3)


In [12]:
df31 = pd.read_csv(path+'0901ClaimRealCOVID-19_tweets_replies_content.csv')
df32 = pd.read_csv(path+'0901NewsFakeCOVID-19_tweets_replies_content.csv')
df33 = pd.read_csv(path+'0901NewsRealCOVID-19_tweets_replies_content.csv')
df32['label'] = 0
df31['label'] = df33['label'] = 1

#combine dataframes
df_sept_reply = pd.concat([df31,df32,df33]).reset_index(drop=True)
#shuffle
df_sept_reply = df_sept_reply.sample(frac=1)  
#only keep tweet_reply and label columns
df_sept_reply = df_sept_reply.drop(['tweet_id', 'tweet_text'], axis=1)
#change df_reply column name to tweet_text in order to combine df_reply with df_tweets 
df_sept_reply = df_sept_reply.rename(columns={"tweet_reply": "tweet_text"})
# print("All Tweets Replies shape: ", df_sept_reply.shape)

df_Sept_Reply = pd.concat([df_Sept_Tweet,df_sept_reply]).reset_index(drop=True)
print('Sept Tweets & Replies shape: ',df_Sept_Reply.shape)

Sept Tweets & Replies shape:  (1302, 3)


In [13]:
df34 = pd.read_csv(path+'1101ClaimRealCOVID-19_tweets_replies.csv')
df35 = pd.read_csv(path+'1101NewsFakeCOVID-19_tweets_replies.csv')
df36 = pd.read_csv(path+'1101NewsRealCOVID-19_tweets_replies.csv')
df35['label'] = 0
df34['label'] = df36['label'] = 1

#combine dataframes
df_nove_reply = pd.concat([df34,df35,df36]).reset_index(drop=True)
#shuffle
df_nove_reply = df_nove_reply.sample(frac=1)  
#only keep tweet_reply and label columns
df_nove_reply = df_nove_reply.drop(['tweet_id', 'tweet_text'], axis=1)
#change df_reply column name to tweet_text in order to combine df_reply with df_tweets 
df_nove_reply = df_nove_reply.rename(columns={"tweet_reply": "tweet_text"})

df_Nove_Reply = pd.concat([df_Nove_Tweet,df_nove_reply]).reset_index(drop=True)
print('Nove Tweets & Replies shape: ',df_Nove_Reply.shape)

Nove Tweets & Replies shape:  (673, 3)


In [14]:
CoAID_tweet = pd.concat([df_May_Tweet,df_July_Tweet,df_Sept_Tweet,df_Nove_Tweet]).reset_index(drop=True)
print(CoAID_tweet.shape)

(149824, 3)


In [15]:
CoAID_tweet_reply = pd.concat([df_May_Reply,df_July_Reply,df_Sept_Reply,df_Nove_Reply]).reset_index(drop=True)
print(CoAID_tweet_reply.shape)

(266677, 3)


In [18]:
CoAID_news_articles = pd.concat([df_May_News,df_July_News,df_Sept_News,df_Nove_News]).reset_index(drop=True)
print(CoAID_news_articles.shape)

(4440, 17)


Convert a list of sentences to a list of lists containing tokenized words

In [19]:
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer

def docs_preprocessor(docs):
    tokenizer = RegexpTokenizer(r'\w+') # Tokenize the words.
    
    for idx in range(len(docs)):
        docs[idx] = docs[idx].lower()  # Convert to lowercase.
        docs[idx] = tokenizer.tokenize(docs[idx])  # Split into words.

    # Remove numbers, but not words that contain numbers.
#     docs = [[token for token in doc if not token.isdigit()] for doc in docs]
    
    # Remove words that are only one character.
    docs = [[token for token in doc if len(token) > 3] for doc in docs]
    
    # Lemmatize all words in documents.
    lemmatizer = WordNetLemmatizer()
    docs = [[lemmatizer.lemmatize(token) for token in doc] for doc in docs]
  
    return docs

In [20]:
%time
CoAID_tweet['texts_tokenized'] = docs_preprocessor(CoAID_tweet['tweet_text'].values)
CoAID_tweet_reply['texts_tokenized'] = docs_preprocessor(CoAID_tweet_reply['tweet_text'].values)
CoAID_news_articles['texts_tokenized'] = docs_preprocessor(CoAID_news_articles['content'].values)

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 9.3 µs


In [22]:
true_CoAID_tweet = []
mis_CoAID_tweet = []
for row in CoAID_tweet.values:
  if row[2] == 1:
    true_CoAID_tweet.append(row[3])
  if row[2] == 0:
    mis_CoAID_tweet.append(row[3])

print(len(true_CoAID_tweet))
print(len(mis_CoAID_tweet))

140126
9698


In [23]:
true_CoAID_tweet_reply = []
mis_CoAID_tweet_reply = []
for row in CoAID_tweet_reply.values:
  if row[2] == 1:
    true_CoAID_tweet_reply.append(row[3])
  if row[2] == 0:
    mis_CoAID_tweet_reply.append(row[3])

print(len(true_CoAID_tweet_reply))
print(len(mis_CoAID_tweet_reply))

250750
15927


In [25]:
true_CoAID_news = []
mis_CoAID_news = []
for row in CoAID_news_articles.values:
  if row[10] == 1:
    true_CoAID_news.append(row[17])
  if row[10] == 0:
    mis_CoAID_news.append(row[17])

print(len(true_CoAID_news))
print(len(mis_CoAID_news))

4024
416


In [27]:
def count_frequency(data):
  #remove stop words first
  stop_words = set(stopwords.words('english')) 
  filtered_texts_tokenized = []
  for text in data:
      filtered_text = []
      for word in text:
        if word not in stop_words:
          filtered_text.append(word)
      filtered_texts_tokenized.append(filtered_text)

  word_dict = {}
  for text in filtered_texts_tokenized:
    for word in text:
      if word not in word_dict:
        word_dict[word]=1
      else:
        word_dict[word]+=1
  #sort dictionary 
  word_dict = OrderedDict(sorted(word_dict.items(), key=lambda t: t[1],reverse = True))
  i=0
  print("Word ------ Frequency")
  for word,frequency in word_dict.items():
    print(word," ------ ",frequency)
    i+=1
    if i==20:
      break

In [28]:
count_frequency(true_CoAID_tweet)

Word ------ Frequency
http  ------  130083
covid  ------  60918
coronavirus  ------  31474
people  ------  14189
health  ------  12322
case  ------  9372
home  ------  8901
covid19  ------  8603
help  ------  8110
disease  ------  8001
spread  ------  7787
resource  ------  7773
virus  ------  7699
testing  ------  7694
need  ------  7015
mask  ------  6955
test  ------  6891
community  ------  6178
pandemic  ------  6074
symptom  ------  5690


In [29]:
count_frequency(mis_CoAID_tweet)

Word ------ Frequency
http  ------  9833
coronavirus  ------  6711
covid  ------  2051
test  ------  1076
bill  ------  939
gate  ------  938
confirms  ------  919
death  ------  898
health  ------  890
positive  ------  873
bioweapon  ------  820
million  ------  763
say  ------  709
trump  ------  706
vaccine  ------  702
virus  ------  660
patient  ------  649
zero  ------  639
scientist  ------  634
china  ------  593


In [30]:
count_frequency(true_CoAID_tweet_reply)

Word ------ Frequency
http  ------  156004
covid  ------  69434
coronavirus  ------  38176
people  ------  27950
health  ------  16670
case  ------  14634
virus  ------  13466
need  ------  13000
home  ------  12444
covid19  ------  12251
like  ------  12009
mask  ------  11890
test  ------  11439
help  ------  11400
testing  ------  11093
time  ------  11011
know  ------  10913
spread  ------  10237
disease  ------  10074
death  ------  9182


In [31]:
count_frequency(mis_CoAID_tweet_reply)

Word ------ Frequency
http  ------  11120
coronavirus  ------  7404
covid  ------  2307
test  ------  1153
gate  ------  1118
death  ------  1108
bill  ------  1091
virus  ------  1058
health  ------  1035
people  ------  1018
trump  ------  996
confirms  ------  931
positive  ------  924
million  ------  881
china  ------  860
vaccine  ------  851
bioweapon  ------  836
like  ------  824
say  ------  785
news  ------  715


In [32]:
count_frequency(true_CoAID_news)

Word ------ Frequency
covid  ------  3632
people  ------  1419
coronavirus  ------  1328
service  ------  1305
protect  ------  1259
health  ------  1217
online  ------  1204
using  ------  1204
data  ------  1132
website  ------  1121
attack  ------  1109
security  ------  1100
disease  ------  1001
2020  ------  829
information  ------  805
pandemic  ------  783
virus  ------  761
time  ------  716
based  ------  643
state  ------  642


In [33]:
count_frequency(mis_CoAID_news)

Word ------ Frequency
coronavirus  ------  157
facebook  ------  156
covid  ------  55
health  ------  51
virus  ------  51
people  ------  47
like  ------  46
president  ------  46
vaccine  ------  44
china  ------  42
death  ------  40
want  ------  40
state  ------  40
trump  ------  39
world  ------  38
said  ------  33
time  ------  33
case  ------  31
pandemic  ------  30
outbreak  ------  29
